In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')

import numpy as np
import scipy as sp
import pandas as pd
import numpy.linalg as la
import numpy.random as rand
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy.stats as stats


import time
import pickle


import noise_models as noise
import model_fitting as fit
import fig_plot as fplot
import thermo_models as thermo



sns.set_theme(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':False})

# Load Data

In [ ]:
seed = 42

print("Seed:", seed)

rand.seed(seed)


# Name of dataset folder
label = "220520_pushpull"

components = ["phospho", "substrate", "kinase", 'pptase', 'kinase2', 'kinase2_phospho']


df_dataset_key = pd.read_csv("../data/"+label+"/dataset_key.csv", sep='\s*,\s*', engine='python').set_index("exp_name")
# df_dataset_key = df_dataset_key.query("exp_name=='kinase_zipper_L+A'")
display(df_dataset_key)

# df_dataset_key = df_dataset_key.query("exp_name=='substrate_only' or exp_name=='kinase_zipper_L+A'")

df_MOCU_key = pd.read_csv("../data/"+label+"/MOCU_key.csv", sep='\s*,\s*', engine='python').set_index("component")
display(df_MOCU_key)


# load datasets

df_list = []
for exp_name, row in df_dataset_key.iterrows():
    
    df = pd.read_csv("../data/{}/{}.csv".format(label, row['file_name']))
    df = df.drop("Unnamed: 0", axis=1, errors='ignore').sample(frac=1.0, replace=False, random_state=seed).reset_index(drop=True)
#     df = df.drop("Unnamed: 0", axis=1, errors='ignore').sample(n=200, replace=False, random_state=seed).reset_index(drop=True)

    df = df.rename(columns={row['substrate_col']:'substrate_anti_exp', 
                         row['phospho_col']:'phospho_anti_exp', 
                         row['kinase_col']:'kinase_anti_exp'})
    
    if row['model'] == 'pushpull' or row['model'] == 'two_layer' or row['model'] == 'two_layer_nowriter' or row['model'] == 'two_layer_noeraser':
        df = df.rename(columns={row['pptase_col']:'pptase_anti_exp'})
    else:
        df['pptase_anti_exp'] = 1e-8
        
    
    if row['model'] == 'two_layer' or row['model'] == 'two_layer_nowriter' or row['model'] == 'two_layer_noeraser':
        df = df.rename(columns={row['kinase2_col']:'kinase2_anti_exp'})
        df = df.rename(columns={row['kinase2_phospho_col']:'kinase2_phospho_anti_exp'})
        df['kinase2_phospho_anti_exp'] = df['kinase2_phospho_anti_exp']

    else:
        df['kinase2_anti_exp'] = 1e-8
        df['kinase2_phospho_anti_exp'] = 1e-8
        
   
    df.drop(df.columns.difference(['substrate_anti_exp','phospho_anti_exp', 'kinase_anti_exp', 'pptase_anti_exp', 'kinase2_anti_exp', 'kinase2_phospho_anti_exp']), axis=1, inplace=True)
    
    df['exp_name'] = exp_name
    df.index.rename('cell_index', inplace=True)
    
    
    df_list.append(df)
    
# dataframe containing all datasets   
df_data = pd.concat(df_list) #.drop("Unnamed: 0", axis=1, errors='ignore')
df_data = df_data.reset_index().set_index(['cell_index', 'exp_name'])
df_data = df_data.reorder_levels(df_data.index.names[::-1])

print(len(df_data.index))
df_data.dropna(inplace=True)
print(len(df_data.index))


for exp_name, group in df_data.groupby('exp_name'):
    print(exp_name)
    
    print(len(group), len(group[(group[group.columns] > 0.0).all(axis=1)]))


df_data = df_data[(df_data[df_data.columns] > 0.0).all(axis=1)]
print(len(df_data.index))




display(df_data)

# Plot raw data for each experiment

In [ ]:
for exp_name, row in df_dataset_key.iterrows():
    
    if row['model'] == 'push' or row['model'] == "substrate_only" or row['model'] == "non-pplatable":
        fplot.plot_push_dataset_summary(df_data, exp_name)
    elif row['model'] == 'pushpull' or row['model'] == 'two_layer' or row['model'] == 'two_layer_nowriter':
        fplot.plot_pushpull_dataset_summary(df_data, exp_name)
                

    

# Setup noise models

In [ ]:
# setup noise model dictionary
noise_models = {c:dict() for c in components}
print(noise_models)

try:
    with open("../data/"+label+"/noise_model_params.pkl", 'rb') as pkl_file:
        noise_model_params = pickle.load(pkl_file)
except:
    noise_model_params = {}
    
    
    
display(noise_model_params)

# Noise models for converting antibody to GFP units

Conversion takes place by fitting a linear regression model.

$\log(anti) = A \log(GFP) + B$

In [ ]:
# points per bin
ppbin = 100

for c in components:
    
    print(c)
    
    # distribution of antibodies and GFP for non-empty cells
    df = pd.read_csv("../data/{}/{}.csv".format(label, df_MOCU_key.loc[c, 'file_name']))    
    anti = df[df_MOCU_key.loc[c, 'anti_col_name']].values
    GFP = df[df_MOCU_key.loc[c, 'GFP_col_name']].values
    print(len(anti), len(GFP))
    idx = (anti > 0.0) & (GFP > 0.0)
    anti = anti[idx]
    GFP = GFP[idx]
    
    print(len(anti), len(GFP))
    
    noise_models[c]['anti'] = noise.BackgroundDist(anti, ppbin=ppbin)
    noise_models[c]['GFP'] = noise.BackgroundDist(GFP, ppbin=ppbin)
    
    # linear mode for converting antibody to GFP measurements
    noise_models[c]['anti2GFP'] = noise.LinearNoise(anti, GFP)
    
    
    # distribution of antibodies and GFP for empty cells
    df = pd.read_csv("../data/{}/{}.csv".format(label, df_MOCU_key.loc['empty_'+c, 'file_name']))
    anti = df[df_MOCU_key.loc['empty_'+c, 'anti_col_name']].values
    GFP = df[df_MOCU_key.loc['empty_'+c, 'GFP_col_name']].values
    print(len(anti), len(GFP))
    idx = (anti > 0.0) & (GFP > 0.0)
    anti = anti[idx]
    GFP = GFP[idx]
    print(len(anti), len(GFP))
    
    noise_models[c]['anti_background'] = noise.BackgroundDist(anti, ppbin=ppbin)
    noise_models[c]['GFP_background'] = noise.BackgroundDist(GFP, ppbin=ppbin)

    
    # convert antibody background to GFP units
    empty_anti_as_GFP = noise_models[c]['anti2GFP'].transform(noise_models[c]['anti_background'].get_data())       
    
    noise_models[c]['anti_as_GFP_background'] = noise.BackgroundDist(empty_anti_as_GFP, ppbin=ppbin)
    
    # lognormal noise model with background
    noise_models[c]['GFP2MOCU'] = noise.LogNormalBGNoise(noise_models[c]['anti_as_GFP_background'])
        
    

        
    fig, axes = plt.subplots(1, 3, figsize=(12, 4), squeeze=False)
    
    fig.suptitle(c, y=1.05)
    
    ax = axes[0, 0]
    
    sns.histplot(x=noise_models[c]['anti_background'].get_data(), y=noise_models[c]['GFP_background'].get_data(), 
                              bins=(100, 100), color='g',
                         log_scale=(True, True), ax = ax)
    
    noise_models[c]['anti2GFP'].plot(ax)
    
    ax.set_title(c)
    
    ax.set_ylabel('out:GFP')
    ax.set_xlabel('in:antibody')
    
    ax.set_xlim(1e-1, 1e5)
    ax.set_ylim(1e0, 1e6)
    

    ax = axes[0, 1]
    
    noise_models[c]['anti_background'].plot(ax, color='g')
    noise_models[c]['anti'].plot(ax, color='b')
    
    sns.histplot(noise_models[c]['anti_background'].get_data(), binrange=(-1, 5), log_scale=True, bins=64, ax=ax, 
                 label='background', fill=True, color='g', stat='density')
    sns.histplot(noise_models[c]['anti'].get_data(), binrange=(-1, 5), log_scale=True, bins=64, ax=ax, 
                 label='signal', fill=True, color='b', stat='density')
  


    ax.set_xscale('log')
    ax.set_xlabel("Antibody")

    ax.set_xlim(1e-1, 1e5)


    ax = axes[0, 2]
    
    noise_models[c]['anti_as_GFP_background'].plot(ax, color='g')
    noise_models[c]['GFP'].plot(ax, color='b')

    sns.histplot(noise_models[c]['GFP_background'].get_data(), binrange=(0, 6), log_scale=True, bins=64, ax=ax, 
                 label='GFP background', fill=False, color='grey', stat='density', element='step')
    sns.histplot(noise_models[c]['anti_as_GFP_background'].get_data(), binrange=(0, 6), log_scale=True, bins=64, ax=ax, 
                 label='antibody background', fill=True, color='g', stat='density')
    sns.histplot(noise_models[c]['GFP'].get_data(), binrange=(0, 6), log_scale=True, bins=64, ax=ax, 
                 label='signal', fill=True, color='b', stat='density')

    
    ax.set_xscale('log')
    ax.set_xlabel("GFP")
    
    ax.legend(loc='lower left', fontsize='xx-small', bbox_to_anchor=(0.0, 1.0), ncol=1)

    ax.set_xlim(1e0, 1e6)

    plt.show()
    


# Convert antibody measurements to GFP units

In [ ]:
for c in components:
    df_data[c+'_GFP_infer'] = 0.0
    
for exp_name, row in df_dataset_key.iterrows():
    
    print(exp_name)
        
    df_tmp = df_data.query("exp_name==@exp_name")
    
    for c in components:

        # a weird way to check for nans or empty values
        if row[c+'_col'] != row[c+'_col']:
            continue
            

        df_data.loc[df_tmp.index, c+'_GFP_infer'] = noise_models[c]['anti2GFP'].transform(df_data.loc[df_tmp.index, c+'_anti_exp'])
        
        
display(df_data)

# Construct noise model to remove background


In [ ]:
for exp_name, row in df_dataset_key.iterrows():
    
    print(exp_name)
        
    df_tmp = df_data.query("exp_name==@exp_name")
    
    if exp_name not in noise_model_params:
        noise_model_params[exp_name] = {}
    
    for c in components:
        
        if c == 'phospho' or c == 'kinase2_phospho':
            continue

        # a weird way to check for nans or empty values
        if row[c+'_col'] != row[c+'_col']:
            continue
            
            
        bg_noise = noise_models[c]['anti_as_GFP_background']
        MOCU_noise = noise_models[c]['GFP2MOCU']
        
        GFP = df_data.loc[df_tmp.index, c+'_GFP_infer'] 
        
        # fit background noise model if doesn't exist
        if c not in noise_model_params[exp_name]:

            params = fit.fit_bg_noise(GFP, MOCU_noise) 
  
        
            (mu, sigma) = params.tolist()
            c0 = np.exp(mu)
        
            noise_model_params[exp_name][c] = (c0, sigma)

        
        
        
        (c0, sigma) = noise_model_params[exp_name][c]   

        x = np.logspace(0, 6, 1000, base=10)

        y = MOCU_noise.calc_prob_meas(x, c0*np.ones_like(GFP), sigma)*x*np.log(10)

        fig, axes = plt.subplots(1, 2, figsize=(10, 4), squeeze=False)

        ax = axes[0, 0]
        
        
        ax.set_xscale('log')


        sns.histplot(bg_noise.get_data(), binrange=(0, 6), log_scale=True, bins=128, ax=ax, 
                             label='control', color='g', stat='density', element='step')

        bg_noise.plot(ax)

        sns.histplot(GFP, binrange=(0, 6), log_scale=True, bins=128, ax=ax, 
                             label='control', color='b', stat='density')
        
        
        ylim = ax.get_ylim()
        ax.plot(x, y, 'k-', ms=3.0)
        
        ax.set_ylim(*ylim)
        
        ax.set_title(c)

        
        ax = axes[0, 1]

        x = np.logspace(0, 6, base=10, num=100)

        MOCU = MOCU_noise.cal_mean_conc(x, c0*np.ones_like(x), sigma)

        ax.plot(x, MOCU, 'b-', label='MOCU')
        ax.plot(x, x-MOCU, 'g-', label='bg')

        ax.plot(x, x, 'k--')

        ax.plot(x, x/2, 'r--')


        ax.vlines([bg_noise.get_data().mean(), bg_noise.bin_edges[0], bg_noise.bin_edges[-1]], ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], ls='--', color='k')

        ax.hlines(bg_noise.get_data().mean(), xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], ls='--', color='k')

        ax.set_xscale('log')
        ax.set_yscale('log')


        ax.set_title(c)

        ax.set_ylabel('MOCU')
        ax.set_xlabel('GFP')

        ax.legend()
        
        fig.suptitle(exp_name, y=1.05)

        plt.show()
        
    
                
        

# Save noise params

In [ ]:
display(noise_model_params)

with open("../data/"+label+"/noise_model_params.pkl", 'wb') as pkl_file:
    pickle.dump(noise_model_params, pkl_file)
    

# Transform to MOCU

In [ ]:
for c in components:
    
    if c == 'phospho' or c == 'kinase2_phospho':
            continue
    
    df_data[c+'_MOCU_infer'] = 0.0

for exp_name, row in df_dataset_key.iterrows():
    
    print(exp_name)
        
    df_tmp = df_data.query("exp_name==@exp_name")
    
    for c in components:
        
        if c == 'phospho' or c == 'kinase2_phospho':
            continue

        # a weird way to check for nans or empty values
        if row[c+'_col'] != row[c+'_col']:
            continue
            
        MOCU_noise = noise_models[c]['GFP2MOCU']
        
        (c0, sigma) = noise_model_params[exp_name][c]
            
        GFP = df_data.loc[df_tmp.index, c+'_GFP_infer'] 
        
        MOCU = MOCU_noise.cal_mean_conc(GFP, c0*np.ones_like(GFP), sigma)
        
        df_data.loc[df_tmp.index, c+'_MOCU_infer']  = MOCU
        
        df_data.loc[df_tmp.index, c+'_GFP_denoise'] = MOCU + noise_models[c]['anti_as_GFP_background'].mean

display(df_data)

# Save Inferred MOCU Values

In [ ]:
df_data.to_csv("../data/"+label+"/model_predictions.csv", sep=',')

In [ ]:


for exp_name, row in df_dataset_key.iterrows():
    
    df_tmp = df_data.query("exp_name==@exp_name")
    
    binrange = (0, 6)
        
    for c in components:
        
        # a weird way to check for nans or empty values
        if row[c+'_col'] != row[c+'_col']:
            continue
        
        fig, axes = plt.subplots(1, 1, figsize=(6, 4), squeeze=False)
    
        fig.suptitle(exp_name+": " + c)

        ax = axes[0, 0]

        sns.histplot(noise_models[c]['anti_as_GFP_background'].get_data(), binrange=binrange, log_scale=True, bins=64, ax=ax, 
                     label='background', color='g', stat='density')
        sns.histplot(noise_models[c]['GFP'].get_data(), binrange=binrange, log_scale=True, bins=64, ax=ax, 
                     label='control', color='b', stat='density')
        sns.histplot(df_tmp[c+"_GFP_infer"], binrange=binrange, log_scale=True, bins=64, ax=ax, 
                     label='inferred GFP', element='step', fill=False, color='k', stat='density')

        ylim = ax.get_ylim()
        
        if c != 'phospho' and c != 'kinase2_phospho':
            sns.histplot(df_tmp[c+"_GFP_denoise"], binrange=binrange, log_scale=True, bins=64, ax=ax, 
                         label='MOCU', element='step', fill=False, color='r', stat='density')

            ax.set_ylim(*ylim)
            
        ax.set_xscale('log')
        ax.set_xlabel("GFP")
        
        ax.legend(loc='upper right', fontsize='xx-small', title=c)
            



        plt.tight_layout()
        plt.show()
        

                

    

In [ ]:

# df = pd.read_csv("../data/{}/{}.csv".format(label, df_MOCU_key.loc['empty_phospho', 'file_name']))    
# phospho_anti = df[df_MOCU_key.loc['empty_phospho', 'anti_col_name']].values

# df = pd.read_csv("../data/{}/{}.csv".format(label, df_MOCU_key.loc['empty_substrate', 'file_name']))    
# substrate_anti = df[df_MOCU_key.loc['empty_substrate', 'anti_col_name']].values

# print(len(phospho_anti), len(substrate_anti))

# idx = (phospho_anti > 0) & (substrate_anti > 0)


# sns.histplot(phospho_anti, binrange=(-200, 200))

# plt.show()


# sns.histplot(substrate_anti, binrange=(-200, 200))

# plt.show()


# phospho_anti = phospho_anti[idx]
# substrate_anti = substrate_anti[idx]

# print(len(phospho_anti))


# df = pd.read_csv("../data/{}/{}.csv".format(label, df_MOCU_key.loc['empty_kinase', 'file_name']))    
# kinase_anti = df[df_MOCU_key.loc['empty_kinase', 'anti_col_name']].values

# sns.histplot(kinase_anti, binrange=(-200, 200))

# plt.show()

for exp_name, row in df_dataset_key.iterrows():
    
    df_tmp = df_data.query("exp_name==@exp_name")
            
    fig, axes = plt.subplots(1,3, constrained_layout=True, figsize=(12, 5), squeeze=False)
    ax = axes[0, 0]
    
#     sns.histplot(x=substrate_anti, y=phospho_anti, ax=ax, log_scale=(True, True), color='g')
    
    sns.histplot(df_tmp, x='substrate_anti_exp', y='phospho_anti_exp', ax=ax, log_scale=(True, True))
    ax.plot([1e-1, 1e5], [1e-1, 1e5], 'k--')
    
    ax.set_xlim(1e-1, 1e5)
    ax.set_ylim(1e-1, 1e5)
    
    ax = axes[0, 1]
    sns.histplot(df_tmp, x='substrate_GFP_infer', y='phospho_GFP_infer', ax=ax, log_scale=(True, True))
    ax.plot([1e0, 1e6], [1e0, 1e6], 'k--')
    
    ax.set_xlim(1e0, 1e6)
    ax.set_ylim(1e0, 1e6)
    
    ax = axes[0, 2]
    sns.histplot(df_tmp, x='substrate_GFP_denoise', y='phospho_GFP_infer', ax=ax, log_scale=(True, True))
    ax.plot([1e0, 1e6], [1e0, 1e6], 'k--')
    
    ax.set_xlim(1e0, 1e6)
    ax.set_ylim(1e0, 1e6)
    
    fig.suptitle(exp_name)
    
    plt.show()
    
    
                

    ###### add backgrounds dists to each of these plots!

# Load prefit parameters

In [ ]:
try:
    df_params = pd.read_csv("../data/"+label+"/model_params.csv", sep=',', engine='python')   
except:
    df_params = None

# Uncomment this to overwrite all previous fits
df_params = None
    
display(df_params)

# Fit all datasets simultaneously (except two layer)

In [ ]:

df_params = fit.fit(df_dataset_key, df_data, df_params=df_params, noise_models=noise_models)

# df_params['val_min'] = 0.0
# df_params['val_max'] = 0.0
# df_params = fit.calc_error(df_dataset_key.query("model!='two_layer'"), df_data, df_params=df_params, noise_models=noise_models)

display(df_params)

# Save model parameters


In [ ]:
df_params.to_csv("../data/"+label+"/model_params.csv", sep=',', index=False)

# Calculate model predictions

In [ ]:

prefit_params, param_to_index, dataset_to_params, x0, bounds = fit.setup_model_params(df_dataset_key, df_params=df_params, noise_models=noise_models)

x = np.zeros_like(x0)
for p in param_to_index:
    x[param_to_index[p]] = df_params.query("name==@p").iloc[0]['val']

args = (df_dataset_key, df_data, prefit_params, param_to_index, dataset_to_params, noise_models)

fit.predict(x, args, df_data)

display(df_data)

print(len(df_data))
print(len(df_data.dropna()))


# Convert predicted MOCU values to antibody values

In [ ]:
df_data['phospho_GFP_denoise'] = 0.0
df_data['phospho_GFP_predict'] = 0.0
df_data['phospho_anti_predict'] = 0.0


for exp_name, row in df_dataset_key.iterrows():
    

    df_tmp = df_data.query("exp_name==@exp_name")
    
    phospho_sigma = df_params.query("name=='phospho_sigma'")['val'].values[0]

    MOCU = df_data.loc[df_tmp.index, 'phospho_MOCU_predict']
    
    df_data.loc[df_tmp.index, 'phospho_GFP_denoise'] = MOCU + noise_models['phospho']['anti_as_GFP_background'].mean
        
    df_data.loc[df_tmp.index, 'phospho_GFP_predict'] = noise_models['phospho']['GFP2MOCU'].sample(MOCU, phospho_sigma)

    df_data.loc[df_tmp.index, 'phospho_anti_predict'] = noise_models['phospho']['anti2GFP'].inverse_transform(df_data.loc[df_tmp.index, 'phospho_GFP_predict'])

    
display(df_data)

# Save predictions

In [ ]:
df_data.to_csv("../data/"+label+"/model_predictions.csv", sep=',')

In [ ]:
for exp_name, row in df_dataset_key.iterrows():

    
    df_tmp = df_data.query("exp_name==@exp_name")
            
    fig, axes = plt.subplots(1,3, constrained_layout=True, figsize=(12, 5), squeeze=False)
    ax = axes[0, 0]
    
    sns.histplot(df_tmp, x='substrate_anti_exp', y='phospho_anti_exp', ax=ax, log_scale=(True, True), binrange=[[0, 6], [0, 6]], bins=100)
    ax.plot([1e0, 1e6], [1e0, 1e6], 'k--')
    
    ax.set_xlim(1e0, 1e6)
    ax.set_ylim(1e0, 1e6)
    
    ax = axes[0, 1]
    sns.histplot(df_tmp, x='substrate_GFP_infer', y='phospho_GFP_infer', ax=ax, log_scale=(True, True), binrange=[[0, 6], [0, 6]], bins=100)
    ax.plot([1e0, 1e6], [1e0, 1e6], 'k--')
#     ax.plot([1e0, 1e6], [2*1e0, 2*1e6], 'r--')
    sns.histplot(df_tmp, x='substrate_GFP_denoise', y='phospho_GFP_denoise', ax=ax, log_scale=(True, True), color='g', binrange=[[0, 6], [0, 6]], bins=100)

    ax.set_xlim(1e0, 1e6)
    ax.set_ylim(1e0, 1e6)
    
    ax = axes[0, 2]
    ax.plot([1e0, 1e6], [1e0, 1e6], 'k--')
    
    sns.histplot(df_tmp, x='substrate_GFP_infer', y='phospho_GFP_predict', ax=ax, log_scale=(True, True), color='g', binrange=[[0, 6], [0, 6]], bins=100)
    
    ax.set_xlim(1e0, 1e6)
    ax.set_ylim(1e0, 1e6)
    
    fig.suptitle(exp_name)
    
    plt.show()